In [1]:
############################# Data preparation script for the SNCB Data Challenge #############################

import numpy as np
import pandas as pd

MIN_BEFORE_INCIDENT = 30
MIN_AFTER_INCIDENT = 15
   
# Read CSV file
data = pd.read_csv('sncb_data_challenge.csv', sep=';')

# Convert string to list of integers
col_list_int = ['vehicles_sequence', 'events_sequence','seconds_to_incident_sequence']
for col in col_list_int:
    data[col] = data[col].apply(lambda x: list(map(int, x.strip('[]').split(','))))

# Convert string to list of floats
col_list_float = ['train_kph_sequence']
for col in col_list_float:
    data[col] = data[col].apply(lambda x: list(map(float, x.strip('[]').split(','))))

# Convert string to list
col_list_str = ['dj_ac_state_sequence', 'dj_dc_state_sequence']
for col in col_list_str:
    data[col] = data[col].apply(lambda x: list(map(str, x.strip('[]').split(','))))

# Create a new column for the duration of each event
data['duration_sequence'] = [[] for _ in range(len(data))]

# Compute the acceleration
data['acceleration_seq'] = data.apply(
    lambda row: [
        (row['train_kph_sequence'][i + 1] - row['train_kph_sequence'][i]) / 
        (row['seconds_to_incident_sequence'][i + 1] - row['seconds_to_incident_sequence'][i])
        if (row['seconds_to_incident_sequence'][i + 1] - row['seconds_to_incident_sequence'][i]) != 0 and row['vehicles_sequence'][i+1] == row['vehicles_sequence'][i] else np.nan
        for i in range(len(row['train_kph_sequence']) - 1)
    ], axis=1)

# Remove redundant events
for i in range(len(data['events_sequence'])):
    new_vehicles_sequence = []
    new_events_sequence = []
    new_seconds_to_incident_sequence = []
    new_train_kph_sequence = []
    new_dj_ac_state_sequence = []
    new_dj_dc_state_sequence = []
    new_acceleration_seq = []
    duration_sequence = []
    
    start_index = 0
    # duration = 0
    for j in range(len(data['events_sequence'][i]) - 1):
        is_before_incident = data['seconds_to_incident_sequence'][i][j] >= -MIN_BEFORE_INCIDENT * 60
        is_after_incident = data['seconds_to_incident_sequence'][i][j] <= MIN_AFTER_INCIDENT * 60
        time_condition = is_before_incident and is_after_incident
        # Skip row that are not in the time window
        if not time_condition:
            start_index = j+1
            continue
        
        # Keep every event that is not the same as the previous one
        if j == start_index or data['events_sequence'][i][j] != new_events_sequence[-1]:
            new_vehicles_sequence.append(data['vehicles_sequence'][i][j])
            new_events_sequence.append(data['events_sequence'][i][j])
            new_seconds_to_incident_sequence.append(data['seconds_to_incident_sequence'][i][j])
            new_train_kph_sequence.append(data['train_kph_sequence'][i][j])
            new_dj_ac_state_sequence.append(data['dj_ac_state_sequence'][i][j])
            new_dj_dc_state_sequence.append(data['dj_dc_state_sequence'][i][j])
            if j < len(data['acceleration_seq'][i]):
                new_acceleration_seq.append(data['acceleration_seq'][i][j])
            duration_sequence.append(0)

        # Compute the duration of the event
        elif data['events_sequence'][i][j] == new_events_sequence[-1]:
            duration_sequence[-1] += abs(data['seconds_to_incident_sequence'][i][j] - data['seconds_to_incident_sequence'][i][j-1])
            
    if len(new_events_sequence) == 0:
        continue
    
    data.at[i, 'vehicles_sequence'] = new_vehicles_sequence
    data.at[i, 'events_sequence'] = new_events_sequence
    data.at[i, 'seconds_to_incident_sequence'] = new_seconds_to_incident_sequence
    data.at[i, 'train_kph_sequence'] = new_train_kph_sequence
    data.at[i, 'dj_ac_state_sequence'] = new_dj_ac_state_sequence
    data.at[i, 'dj_dc_state_sequence'] = new_dj_dc_state_sequence
    data.at[i, 'acceleration_seq'] = new_acceleration_seq
    data.at[i, 'duration_sequence'] = duration_sequence

# for i in range(len(data['events_sequence'])):
#     if len(data['events_sequence'][i]) == 0:
#         print("empty")
    
# Save the modified DataFrame to a new CSV file
data.to_csv('sncb_prepared.csv', sep=';', index=False)

In [1]:
############################# Frequent itemsets (FP-Growth) #############################

import pandas as pd
from mlxtend.frequent_patterns import fpgrowth
import ast  # For safely evaluating string representations of lists

MIN_SUPPORT = 0.7

def find_frequent_itemsets_fp_growth(data, min_support=MIN_SUPPORT):
    """
    Finds the most frequent sequences of events for each incident type using FP-Growth.
    """
    # Get all unique incident types
    incident_types = data['incident_type'].unique()
    results = {}

    # Convert stringified lists to actual lists of integers
    data['events_sequence'] = data['events_sequence'].apply(lambda x: ast.literal_eval(x))

    for incident in incident_types:
        # Check if the csv file already exists
        # try:
        #     most_frequent = pd.read_csv(f'results\\results_{incident}.csv', sep=';')
        #     if most_frequent is not None:
        #         results[incident] = most_frequent
        #         continue
        # except:
        #     pass

        # Filter rows for the current incident type
        filtered_data = data[data['incident_type'] == incident]

        # Prepare transactions: each transaction is a sequence of events
        transactions = filtered_data['events_sequence'].tolist()

        # Create a one-hot encoded DataFrame for the events
        unique_events = set(event for sequence in transactions for event in sequence)  # All unique events
        transaction_df = pd.DataFrame([
            {event: (event in sequence) for event in unique_events} for sequence in transactions
        ])
        print(transaction_df)
        # Apply FP-Growth algorithm
        frequent_itemsets = fpgrowth(transaction_df, min_support=min_support, use_colnames=True)

        # Sort by support and keep top results
        if not frequent_itemsets.empty:
            most_frequent = frequent_itemsets.sort_values(by='support', ascending=False)
            most_frequent['itemsets'] = frequent_itemsets['itemsets'].apply(lambda x: list(x))
            results[incident] = most_frequent
        else:
            results[incident] = None
        
        # store the results in a csv file
        most_frequent.to_csv(f'results\\results\\results_{incident}.csv', sep=';', index=False)
    # Run for all the database
    transactions = data['events_sequence'].tolist()
    unique_events = set(event for sequence in transactions for event in sequence)  # All unique events
    transaction_df = pd.DataFrame([
        {event: (event in sequence) for event in unique_events} for sequence in transactions
    ])
    database_frequent_itemsets = fpgrowth(transaction_df, min_support=min_support, use_colnames=True)
    database_frequent_itemsets = database_frequent_itemsets.sort_values(by='support', ascending=False)
    database_frequent_itemsets['itemsets'] = database_frequent_itemsets['itemsets'].apply(lambda x: list(x))
    database_frequent_itemsets.to_csv(f'results\\results\\results_database.csv', sep=';', index=False)
    return results

# Load the dataset
data = pd.read_csv('sncb_prepared.csv', sep=';')

# Run the function
results = find_frequent_itemsets_fp_growth(data)

     1024   3074   1018   4100   3076   3078   2564   3080   1028   3082  ...  \
0   False  False  False   True  False  False  False  False  False   True  ...   
1   False  False  False  False  False  False  False  False  False  False  ...   
2   False  False  False  False  False  False  False  False  False  False  ...   
3   False  False  False   True  False  False  False  False  False  False  ...   
4   False   True  False  False   True   True  False   True  False  False  ...   
..    ...    ...    ...    ...    ...    ...    ...    ...    ...    ...  ...   
73  False  False  False   True  False  False  False  False  False  False  ...   
74  False  False  False   True  False  False  False  False  False  False  ...   
75  False  False  False   True  False  False  False  False  False  False  ...   
76  False  False  False   True  False  False  False  False  False  False  ...   
77  False  False  False  False  False  False  False  False  False  False  ...   

     3054   4082   3056   4

In [3]:
############################# Frequent sequences #############################
# import pandas as pd

# def find_frequent_sequences_fp_growth(data, threshold=0.9):
#     """
#     Finds the most frequent sequences of events for each incident type using GSP.
#     """
#     # Get all unique incident types
#     incident_types = data['incident_type'].unique()
#     results = {}

#     # Convert stringified lists to actual lists of integers
#     data['events_sequence'] = data['events_sequence'].apply(lambda x: ast.literal_eval(x))

#     for incident in incident_types:
#         # Check if the csv file already exists
#         try:
#             most_frequent = pd.read_csv(f'results\\results_{incident}.csv', sep=';')
#             results[incident] = most_frequent
#             continue
#         except:
#             pass

#         # Filter rows for the current incident type
#         filtered_data = data[data['incident_type'] == incident]

#         # Prepare transactions: each transaction is a sequence of events
#         transactions = filtered_data['events_sequence'].tolist()

#         # Create a one-hot encoded DataFrame for the events
#         unique_events = set(event for sequence in transactions for event in sequence)  # All unique events
#         transaction_df = pd.DataFrame([
#             {event: (event in sequence) for event in unique_events} for sequence in transactions
#         ])
        
#         count_list = np.zeros(len(unique_events))
#         for i in range(len(unique_events)):
#             count_list[i] = transaction_df[unique_events[i]].sum()

#         for i in range(len(unique_events)):
#             if count_list[i] < threshold*len(transactions):
#                 # transaction_df.drop(columns=[unique_events[i]], inplace=True)
#                 unique_events.remove(unique_events[i])

#         frequent_sequences = unique_events
#         new_frq_seq = unique_events
#         old_frq_seq = unique_events
#         max_time_diff = 50 # TODO: Define a proper value
#         while len(new_frq_seq) > 0:
#             new_frq_seq = []
#             time_diff = max_time_diff
            
        
        



In [5]:
############################# Final Frequent itemsets #############################
import os
import pandas as pd

itemsets_to_not_add = pd.read_csv('results\\results\\results_database.csv', sep=';')

frequent_itmesets = []
# read all the files in the results folder
for filename in os.listdir('results\\results'):
    if filename == 'results_database.csv' or filename == 'results_frequent.csv':
        continue
    incident_frequent_itemset = pd.read_csv(f'results\\results\\{filename}', sep=';')
    for index, row in incident_frequent_itemset.iterrows():
        if row['itemsets'] not in frequent_itmesets and row['itemsets'] not in itemsets_to_not_add['itemsets']:
            frequent_itmesets.append(row['itemsets'])

frequent_itmesets = pd.DataFrame(frequent_itmesets, columns=['itemsets'])
frequent_itmesets.to_csv('results\\results\\results_frequent.csv', sep=';', index=False)

In [6]:
############################# One hot encoding ############################# 
# import pandas as pd

# # Load the dataset
# data = pd.read_csv('sncb_prepared.csv', sep=';')
# frequent_itmesets = pd.read_csv('results\\results_frequent.csv', sep=';')
# columns = frequent_itmesets['itemsets'].tolist()
# columns.append('target')
# final_data = pd.DataFrame(columns=columns)

# # Apply the one hot encoding
# for index, row in data.iterrows():
    
#     one_hot_encoding = []
#     for itemset in frequent_itmesets['itemsets']:
#         itemset = list(itemset.strip('[]').split(','))
#         events_sequence = list(row['events_sequence'].strip('[]').split(','))
#         if all(event in events_sequence for event in itemset):
#             one_hot_encoding.append(1)
#         else:
#             one_hot_encoding.append(0)
#     one_hot_encoding.append(row['incident_type'])
#     final_data = pd.concat([final_data, pd.DataFrame([one_hot_encoding], columns=columns)], ignore_index=True)

# final_data.to_csv('sncb_final.csv', sep=';', index=False)

In [7]:
############################# One hot encoding unique events #############################
import pandas as pd
import numpy as np

# Load the dataset
data = pd.read_csv('sncb_prepared.csv', sep=';')
data['events_sequence'] = data['events_sequence'].apply(lambda x: list(map(int, x.strip('[]').split(','))))

# Extract unique events
unique_events = sorted({event for seq in data['events_sequence'] for event in seq})
event_to_index = {event: idx for idx, event in enumerate(unique_events)}

# Create a one-hot encoded matrix
ohe_matrix = np.zeros((len(data), len(unique_events)), dtype=int)
for i, seq in enumerate(data['events_sequence']):
    for event in seq:
        ohe_matrix[i, event_to_index[event]] = 1

# Convert to DataFrame
ohe_df = pd.DataFrame(ohe_matrix, columns=[str(event) for event in unique_events])

# Add the target column
ohe_df['target'] = data['incident_type']

# Save the OHE data to CSV
ohe_df.to_csv('unique_events_OHE.csv', sep=';', index=False)

In [8]:
############################# One hot encoding ############################# 
import pandas as pd

# Load data
sets = pd.read_csv('results\\results\\results_frequent.csv', sep=';')
sets = sets['itemsets'].apply(lambda x: list(map(int, x.strip('[]').split(',')))).tolist()

OHE_data = pd.read_csv('unique_events_OHE.csv', sep=';')
unique_events = OHE_data.columns.tolist()
unique_events.remove('target')

# Add new columns for each itemset
new_columns = {str(itemset): 0 for itemset in sets}
new_data = pd.DataFrame(new_columns, index=OHE_data.index)

# Perform one-hot encoding for itemsets
for index, row in OHE_data.iterrows():
    for itemset in sets:
        if all(row[str(event)] == 1 for event in itemset):
            new_data.at[index, str(itemset)] = 1

# Drop unnecessary columns
OHE_data.drop(columns=unique_events, inplace=True)

# Combine original and new data
OHE_data = pd.concat([OHE_data, new_data], axis=1)

# Save to file
OHE_data.to_csv('OHE_frequent.csv', sep=';', index=False)